import pandas as pd
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt
import requests as req
import json
from citipy import citipy

#Observed Trend 1: The average temperature decreases as you travel farther from the equator
    
#Observed Trend 2: Humidity does not have a strong correlation with distance from teh equator

#Observed Trend 3: There is no strong correlation with wind speed and distance from the equator

url = "http://api.openweathermap.org/data/2.5/weather?"
myweatherkey = '383110043a1d03ca838185f17b637183'
units = "imperial"
location_df = pd.DataFrame()
query_url = url + "appid=" + myweatherkey + "&units=" + units + "&q="

#Create list of random latutudes and longitudes
for x in range(600):
    rand_latitude = np.random.uniform(low=-90.000,high=90.000,size=1)
    rand_longitude = np.random.uniform(low=-180.000,high=180.000,size=1)
    random_loc = pd.DataFrame([[rand_latitude,rand_longitude]],columns=['latitude','longitude']).astype(float)
    location_df = location_df.append(random_loc)
my_location_df = location_df.reset_index()

#Create random list of cities
citylist = []
row = 0
while row <600:
    latitude = my_location_df['latitude'][row]
    longitude = my_location_df['longitude'][row]
    city=citipy.nearest_city(latitude,longitude)
    cityname = city.city_name
    country_code = city.country_code
    
    if city not in citylist:
        citylist.append([cityname,country_code,latitude,longitude])
    row += 1

citylist_df = pd.DataFrame(citylist,columns=['cityname','country_code','latitude','longitude'])
city_length = len(citylist_df['cityname'])

#Create empty URL list
url_list = []   
cities = citylist_df['cityname']
cities_list = list(cities.values.flatten())

index = 0
for c in cities_list:
    city = c
    url_id = query_url+str(c)
    print("Record " + str(index+1) + " of " + str(city_length) + " " + str(c))
    print(url_id)
    url_list.append(url_id)  
    index +=1
url_list

weather_json_l = []
for city in url_list:
    response = req.get(city).json()
    if response=={'cod': '404', 'message': 'city not found'}:
       continue
    weather_json_l.append(response)  
weather_json_l

lat_data = [data.get("coord").get("lat") for data in weather_json_l]
temp_data = [data.get("main").get("temp") for data in weather_json_l]
city = [data.get("name") for data in weather_json_l]
humidity = [data.get("main").get('humidity') for data in weather_json_l]
wind_speed = [data.get('wind').get('speed') for data in weather_json_l]
cloudiness = [data.get('clouds').get('all') for data in weather_json_l]

weather_dict = {"Temperature (Fahrenheit)": temp_data, 
                "Latitude": lat_data,
                "city":city,
                "humidity":humidity,
                "wind speed":wind_speed,
                "Clouds":cloudiness}
weather_df = pd.DataFrame(weather_dict)

weather_df.set_index("city", inplace=True)
weather_df.head(1)

# save the Series as a csv file
weather_df.to_csv("Weather_df.csv",index=False)

print(len(weather_df['Clouds']))

if len(weather_df['Clouds'])<500:
    print("Add more locations")
else:
    print("Location count is greather than 500, so you are good to go!")

plt.scatter(x=weather_df["Latitude"], 
            y=weather_df["Temperature (Fahrenheit)"],
            edgecolors = "black",
            marker = "o",
            linewidth = 1.0)         
plt.title("City Latitude vs. Temp (F) (02/26/2018)")
plt.xlabel('City Latitude',fontsize=10)
plt.ylabel('Temperature (Fahrenheit)',fontsize=10)
plt.xlim(-80,100)
plt.ylim(-100,150)        
sea.set()

plt.savefig("TemperatureInWorldCities3.png")

plt.show

plt.scatter(x=weather_df["Latitude"], 
            y=weather_df["humidity"],
            edgecolors = "black",
            marker = "o",
            linewidth = 1.0)         
plt.title("City Latitude vs. Humidity (%) (02/26/2018)")
plt.xlabel('City Latitude',fontsize=8)
plt.ylabel('Humidity (%)',fontsize=8)
plt.xlim(-80,100)
plt.ylim(-20,120)
sea.set()
plt.savefig("HumidityWorldCities1.png")

plt.show

plt.scatter(x=weather_df["Latitude"], 
            y=weather_df["humidity"],
            edgecolors = "black",
            marker = "o",
            linewidth = 1.0)          
plt.title("City Latitude vs. Cloudiness (02/26/2018)")
plt.xlabel('City Latitude',fontsize=8)
plt.ylabel('Cloudiness (%)',fontsize=8)
plt.xlim(-80,100)
plt.ylim(-20,120)
sea.set()
plt.savefig("CloudsWorldCities1.png")

plt.show

plt.scatter(x=weather_df["Latitude"], 
            y=weather_df["wind speed"],
            edgecolors = "black",
            marker = "o",
            linewidth = 1.0)   
plt.title("City Latitude vs. Wind Speed (02/26/2018)")
plt.xlabel('City Latitude',fontsize=8)
plt.ylabel('Wind Speed (mph)',fontsize=8)
plt.xlim(-80,100)
plt.ylim(-5,40)
sea.set()
plt.savefig("WindSpeedWorldCities1.png")

plt.show